In [ ]:
import os
import imgaug.augmenters as iaa
import imgaug as ia
import cv2
import matplotlib.pyplot as plt
import keras
from keras.utils.np_utils import *
import numpy as np
from sklearn.model_selection import train_test_split
import cv2
from imblearn.over_sampling import BorderlineSMOTE 
from collections import Counter
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization, Activation, Add, AveragePooling2D, Flatten
from keras.models import Model, Sequential

In [ ]:
root_dir = '/kaggle/input/housetreeperson-sentiment'
class_map = {'alhh':0, 'alhl':1}
num_classes = 2
img_size = 32
batch_size = 32
epochs = 100
encoded_shape = 8*8*4

In [ ]:
# transparent to white
def to_white(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
    # aplha
    width = img.shape[0]
    height = img.shape[1]
    for w in range(width):
        for h in range(height):
            if (img[w,h,3] == 0):
                img[w,h] = [255,255,255,255]
    return img[:,:,:-1]

In [ ]:
# load the dataset
X = []
y = []

for class_name in os.listdir(root_dir):
    for img_name in os.listdir(root_dir+'/'+class_name):
        label = class_map[class_name]
        img = to_white(root_dir+'/'+class_name+'/'+img_name)
#         img = cv2.imread(root_dir+'/'+class_name+'/'+img_name)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (img_size, img_size))
        X.append(img)
        y.append(label)
        
X = np.array(X, dtype=np.uint8)
y = np.array(y)

# y = to_categorical(y, num_classes)

(X_train, X_valid, y_train, y_valid) = train_test_split(X, y, test_size=0.2, stratify=y)
print(f'X_train shape: {X_train.shape}, y_train shape: {y_train.shape}')
print(f'X_valid shape: {X_valid.shape}, y_valid shape: {y_valid.shape}')

In [ ]:
# augumentation 113 * 21
ia.seed(1)

seq = iaa.Sequential([
    iaa.Fliplr(0.5),
    iaa.Crop(percent=(0,0.1)),
    iaa.Sometimes(0.5,iaa.GaussianBlur(sigma=(0,0.5))),
    iaa.LinearContrast((0.75,1.5)),
    iaa.Affine(
        scale={"x":(0.8,1.2),"y":(0.8,1.2)},
        translate_percent={"x":(-0.2,0.2), "y":(-0.2,0.2)},
        rotate=(-25,25),
        shear=(-8,8)
    )
], random_order=True)

origin_X = X
origin_y = y
for i in range(20):
    X = np.append(X, seq(images=origin_X), axis=0)
    y = np.append(y, origin_y, axis=0)
print(X.shape, y.shape)

In [ ]:
print(np.sum(y))

In [ ]:
X = X.astype('float32')/255.

In [ ]:
# # convolutional autoencoder for dimensionality reduction
# input_img = Input(shape=(img_size,img_size,3))
# encoder=Conv2D(16,(3,3),padding='same',activation='relu')(input_img)
# encoder=MaxPooling2D((2,2))(encoder)
# encoder=Conv2D(8,(3,3),padding='same',activation='relu')(encoder)
# encoder=MaxPooling2D((2,2))(encoder)
# encoder=Conv2D(4,(3,3),padding='same',activation='relu')(encoder)
# encoder_out=MaxPooling2D((2,2))(encoder)
# encoder_model = Model(inputs=input_img, outputs=encoder_out)

# decoder=UpSampling2D((2,2))(encoder_out)                                                                 
# decoder=Conv2D(4,(3,3),padding='same',activation='relu')(decoder) 
# decoder=UpSampling2D((2,2))(decoder)                                                                 
# decoder=Conv2D(8,(3,3),padding='same',activation='relu')(decoder)                                               
# decoder=UpSampling2D((2,2))(decoder)
# decoder=Conv2D(16,(3,3),padding='same',activation='relu')(decoder)
# # to original size
# decoder_out=Conv2D(3, (3, 3), padding='same',activation='sigmoid')(decoder)

# autoencoder=Model(input_img, decoder_out)

# autoencoder.summary()
# autoencoder.compile(optimizer='adam',loss='binary_crossentropy')

In [ ]:
# autoencoder.fit(X, X,batch_size=batch_size, epochs=epochs)
# encoder_model.save("./encoder.h5")
# X = encoder_model.predict(X)
# X = np.reshape(X, (X.shape[0],-1))

# print(X.shape)

In [ ]:
# process imbalanced data
print('Original dataset shape %s' % Counter(y))
sm = BorderlineSMOTE(random_state=42)
X, y = sm.fit_resample((np.reshape(X, (X.shape[0],-1))), y)
X = np.reshape(X,(X.shape[0],img_size,img_size,3))
print('Resampled dataset shape %s' % Counter(y))
print(X.shape)

In [ ]:
# classification
def identity_block(X, filters):
    f1, f2, f3 = filters
    X_residual = X
    
    X = Conv2D(filters=f1, kernel_size=(1,1))(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    
    X = Conv2D(filters=f2, kernel_size=(3,3), padding='same')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    
    X = Conv2D(filters=f3, kernel_size=(1,1))(X)
    X = BatchNormalization(axis=3)(X)
    
    X = Add()([X, X_residual])
    X = Activation('relu')(X)
    
    return X
    
def convolution_block(X, filters):
    f1, f2, f3 = filters
    X_residual = X
    
    X = Conv2D(filters=f1, kernel_size=(1,1))(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    
    X = Conv2D(filters=f2, kernel_size=(3,3), padding='same')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    
    X = Conv2D(filters=f3, kernel_size=(1,1))(X)
    X = BatchNormalization(axis=3)(X)
    
    X_residual = Conv2D(f3, kernel_size=(1,1))(X_residual)
    X_residual = BatchNormalization(axis=3)(X_residual)
    
    X = Add()([X, X_residual])
    X = Activation('relu')(X)
    
    return X

def create_ResNetLike_model():
    X_input = Input((img_size,img_size,3))
    # X = ZeroPadding2D((3,3))(X_input)
    X = Conv2D(64, (7,7), strides=(2,2), padding='same')(X_input)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3,3), strides=(2,2))(X)
    
    X = convolution_block(X, filters=[64,64,256])
    X = identity_block(X, filters=[64,64,256])
    X = identity_block(X, filters=[64,64,256])
    
    X = convolution_block(X, filters=[128,128,512])
    X = identity_block(X, filters=[128,128,512])
    X = identity_block(X, filters=[128,128,512])
    X = identity_block(X, filters=[128,128,512])
    
    X = convolution_block(X, filters=[256,256,1024])
    X = identity_block(X, filters=[256,256,1024])
    X = identity_block(X, filters=[256,256,1024])
    X = identity_block(X, filters=[256,256,1024])
    X = identity_block(X, filters=[256,256,1024])
    X = identity_block(X, filters=[256,256,1024])
    
    X = convolution_block(X, filters=[512,512,2048])
    X = identity_block(X, filters=[512,512,2048])
    X = identity_block(X, filters=[512,512,2048])
    X = AveragePooling2D((2,2))(X)
    
    X = Flatten()(X)
    X = Dense(1, activation='sigmoid')(X)
    
    model = Model(inputs=X_input, outputs=X)
    model.compile(loss='binary_crossentropy',
        optimizer='adam',
        metrics=['accuracy'])
    print(model.summary())
    
    return model
    
# resnet_model = create_ResNetLike_model()

In [ ]:
# resnet_model.fit(X,y,batch_size=batch_size, epochs=epochs)

In [ ]:
# resnet_model.save("./resnet.h5")

In [ ]:
# import os
# os.chdir('/kaggle/working')
# from IPython.display import FileLink
# # 将本地文件链接到notebook的文档中
# FileLink('resnet.h5')

In [ ]:
from keras.models import Model, Sequential
from keras.layers import Dropout
# create cnn model
# VGG-like
def create_VGGLike_model(input_shape=(img_size,img_size,3)):
    model = Sequential()

    model.add(Conv2D(filters=16,kernel_size=(3,3),padding='Same',activation='relu',input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=16,kernel_size=(3,3),padding='Same',activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(filters=32,kernel_size=(3,3),padding='Same',activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=32,kernel_size=(3,3),padding='Same',activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(filters=64,kernel_size=(3,3),padding='Same',activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=64,kernel_size=(3,3),padding='Same',activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=64,kernel_size=(3,3),padding='Same',activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(2048, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',
        optimizer='adam',
        metrics=['accuracy'])
#     keras.utils.vis_utils.plot_model(model, to_file='cnn_architecture.png', show_shapes=True, show_layer_names=True)
    print(model.summary())
    return model

vgg_model = create_VGGLike_model()

In [ ]:
vgg_model.fit(X,y,batch_size=batch_size, epochs=epochs)

In [ ]:
vgg_model.save('./vgg_model.h5')

In [ ]:
import os
os.chdir('/kaggle/working')
from IPython.display import FileLink
# 将本地文件链接到notebook的文档中
FileLink('vgg_model.h5')

In [ ]:
# visualization
# plt.figure(figsize=(2, 10))
# plt.subplots_adjust(wspace=0, hspace=0)
# idxs = []

# for i in range(10):
# #     idx = np.random.randint(113)
# #     idxs.append(idx)
#     idx=i
#     plt.subplot(10,2,2*i+1)
#     plt.imshow(X[idx]);plt.axis('off')
#     plt.subplot(10,2,2*i+2)
#     plt.imshow(imgs_aug[idx]);plt.axis('off')

# print(idxs)